In [42]:

import os

import joblib
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectPercentile
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

sns.set_style("darkgrid")


In [43]:
def trim_set(frame):
    return frame[((frame['TRB'] > 2) | (frame['AST'] > 5)) & (frame['PTS'] > 15) & (frame['G'] > 5)]
    # return frame[((frame['TRB'] > 5) | (frame['AST'] > 5)) & (frame['PTS'] > 15) & (frame['Win%'] > .5)]

In [44]:
mvp_data_dir = '../Basketball Reference Stat Scraper/Awards/MVP'
player_data_dir = '../Basketball Reference Stat Scraper/player_stats'

In [27]:
full_data = []

for year in range(1980, 2022):
    # print(year)
    player = pd.read_csv(os.path.join(player_data_dir, f'{year}_player_stats.csv'), index_col=0)
    player.Player = player.Player.str.strip('*')

    player = player.fillna(0)
    full_data.append(player)

len(full_data)
train = pd.concat(full_data)
train = train.reset_index(drop=True)
print(len(train))

17683


In [45]:
features_to_predict = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
                       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
                       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%',
                       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
                       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
                       ]

In [29]:
for j in features_to_predict:

    for i in train.Player.unique():
        train.loc[train['Player'] == i, f'{j}_Diff'] = train.loc[train['Player'] == i, j].diff().shift(-1)

    train.loc[train['Player'] == i, 'Years_Exp'] = range(len(train.loc[train['Player'] == i, 'Player']))
    # train.loc[train['Player'] == i,'Years_Exp'] = range(len(train.loc[train['Player'] == i,'Player']))

In [46]:
# train.loc[train['Player'] == 'Michael Jordan','G_Shift'] = train.loc[train['Player'] == 'Michael Jordan','G_Diff'].shift(-1)
train.loc[train['Player'] == 'Michael Jordan', train.columns.str.contains('Diff')]

,Age_Diff,G_Diff,GS_Diff,MP_Diff,FG_Diff,FGA_Diff,FG%_Diff,3P_Diff,3PA_Diff,3P%_Diff,...,TOV%_Diff,USG%_Diff,OWS_Diff,DWS_Diff,WS_Diff,WS/48_Diff,OBPM_Diff,DBPM_Diff,BPM_Diff,VORP_Diff
1675,1.0,-64.0,-75.0,-13.2,-1.9,-1.6,-0.058,0.1,0.4,-0.006,...,-2.5,8.8,-9.3,-3.2,-12.5,-0.053,1.7,0.7,2.4,-6.1
1997,1.0,64.0,75.0,14.9,5.1,9.6,0.025,-0.1,-0.2,0.015,...,-1.4,-0.3,10.9,4.5,15.4,0.087,0.9,0.2,1.1,9.3
2318,1.0,0.0,0.0,0.4,-0.4,-3.4,0.053,0.0,-0.2,-0.050,...,0.5,-4.2,3.3,1.1,4.3,0.061,0.4,1.8,2.2,1.9
2658,1.0,-1.0,-1.0,-0.2,-1.1,-2.2,0.003,0.2,0.6,0.144,...,2.3,-2.0,-0.6,-0.9,-1.4,-0.016,-0.4,-0.8,-1.1,-1.1
3007,1.0,1.0,1.0,-1.2,0.7,1.8,-0.012,0.8,1.8,0.100,...,-2.1,1.6,0.1,-0.9,-0.8,-0.007,0.7,-1.3,-0.7,-0.8
3366,1.0,0.0,0.0,-2.0,-0.5,-1.6,0.013,-0.7,-1.9,-0.064,...,-1.1,-0.8,0.2,1.1,1.3,0.036,-0.2,1.1,0.8,0.2
3753,1.0,-2.0,-2.0,1.8,-0.3,0.3,-0.020,-0.1,0.2,-0.042,...,0.1,-1.2,-2.8,0.2,-2.6,-0.047,-1.7,-0.7,-2.3,-1.6
4145,1.0,-2.0,-2.0,0.5,0.9,3.0,-0.024,0.7,1.6,0.082,...,-0.4,3.0,-0.1,-0.4,-0.5,-0.004,1.5,0.1,1.5,1.0
4528,2.0,-61.0,-61.0,0.0,-2.9,-1.9,-0.084,-0.1,-1.0,0.148,...,-1.4,-1.5,-10.8,-4.1,-14.9,-0.103,-5.1,-2.0,-7.0,-9.1
5330,1.0,65.0,65.0,-1.6,1.4,-1.2,0.084,0.5,1.3,-0.073,...,1.4,0.1,13.0,5.1,18.1,0.150,4.7,1.6,6.3,8.7


In [47]:
# X = 'Years_Exp'
# Y = 'PTS_Diff'

In [48]:
# train.sort_values(Y,ascending=False).head(10)[['Player','Age','Tm','G','Years_Exp','WS',Y]]

In [49]:
# sns.lineplot(data=train[train['Player'] == 'Michael Jordan'],x='Age',y=Y,color='green',label='MJ')
# plt.show()

In [50]:

# # sns.scatterplot(data=trim_set(train),x='Years_Exp',y='WS_Diff',)
# sns.lineplot(data=trim_set(train),x=X,y=Y,color='blue',errorbar=None,label='AVG')
# # sns.lineplot(data=train[train['Player'] == 'Magic Johnson'],x=X,y=Y,color='purple',label='Magic')
# sns.lineplot(data=train[train['Player'] == 'Steve Nash'],x=X,y=Y,color='green',label='Nash')
# # plt.xticks(range(0,21))
#
#
# # plt.yticks(range(-20,21))
# plt.show()

In [51]:
# test = train[train['Year'] == 2023]

In [52]:
df = train.drop(columns=['STAR', 'DPOY_Rank', 'DPOY_First', 'DPOY_Pts Won', 'DPOY_Pts Max', 'DPOY_Share',
                         'DPOY', 'MIP_Rank', 'MIP_First', 'MIP_Pts Won', 'MIP_Pts Max',
                         'MIP_Share', 'MIP', 'MVP_First', 'MVP_Pts Won',
                         'MVP_Pts Max', 'ROTY_Rank', 'ROTY_First',
                         'ROTY_Pts Won', 'ROTY_Pts Max', 'ROTY_Share', 'ROTY', 'SMOTY_Rank',
                         'SMOTY_First', 'SMOTY_Pts Won', 'SMOTY_Pts Max', 'SMOTY_Share',
                         'SMOTY', 'First'], errors='ignore')

In [38]:
df = trim_set(df)

df = df[~df['PTS_Diff'].isna()]

df.rename(columns={"WS/48": "WS_48", 'WS/48_Diff': 'WS_48_Diff'},
          inplace=True)

In [39]:


feature_list = [
    'Age', 'G', 'GS', 'MP', 'FG', 'FGA',
    'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
    'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
    'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
    'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS_48', 'OBPM', 'DBPM', 'BPM', 'VORP'
]

features_to_predict = [
    'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
    '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
    'DRB','TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%',
    '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
    'USG%', 'OWS', 'DWS', 'WS', 'WS_48', 'OBPM', 'DBPM', 'BPM', 'VORP',
]

x = df[feature_list]

for i in features_to_predict:
    y = df.loc[:, f'{i}_Diff', ]

    pipe = Pipeline(

        [('feat_selection', SelectPercentile(percentile=60),),
         ('scaler', MinMaxScaler()),
         ('reg', RandomForestRegressor())]
    )

    pipe.fit(x, y)

    filename = f'All_Stat_Models/{i}.pkl'
    joblib.dump(pipe, filename)
    print(filename)

All_Stat_Models/Age.pkl
All_Stat_Models/G.pkl
All_Stat_Models/GS.pkl
All_Stat_Models/MP.pkl
All_Stat_Models/FG.pkl
All_Stat_Models/FGA.pkl
All_Stat_Models/FG%.pkl
All_Stat_Models/3P.pkl
All_Stat_Models/3PA.pkl
All_Stat_Models/3P%.pkl
All_Stat_Models/2P.pkl
All_Stat_Models/2PA.pkl
All_Stat_Models/2P%.pkl
All_Stat_Models/eFG%.pkl
All_Stat_Models/FT.pkl
All_Stat_Models/FTA.pkl
All_Stat_Models/FT%.pkl
All_Stat_Models/ORB.pkl
All_Stat_Models/DRB.pkl
All_Stat_Models/TRB.pkl
All_Stat_Models/AST.pkl
All_Stat_Models/STL.pkl
All_Stat_Models/BLK.pkl
All_Stat_Models/TOV.pkl
All_Stat_Models/PF.pkl
All_Stat_Models/PTS.pkl
All_Stat_Models/PER.pkl
All_Stat_Models/TS%.pkl
All_Stat_Models/3PAr.pkl
All_Stat_Models/FTr.pkl
All_Stat_Models/ORB%.pkl
All_Stat_Models/DRB%.pkl
All_Stat_Models/TRB%.pkl
All_Stat_Models/AST%.pkl
All_Stat_Models/STL%.pkl
All_Stat_Models/BLK%.pkl
All_Stat_Models/TOV%.pkl
All_Stat_Models/USG%.pkl
All_Stat_Models/OWS.pkl
All_Stat_Models/DWS.pkl
All_Stat_Models/WS.pkl
All_Stat_Models/